# IPL WIN PROBABILITY PREDICTOR

## (1) DATA PREPROCESSING

#### Step 1: Import all the required libraries

In [3]:
import numpy as np
import pandas as pd

#### Step 2: Read Dataset from CSV file

In [6]:
match_df = pd.read_csv('../data/IPL_Matches_2008_2022.csv')
delivery_df = pd.read_csv('../data/IPL_Ball_by_Ball_2008_2022.csv')

In [ ]:
match_df.head()

In [ ]:
delivery_df.head()

##

#### Step 3: Add the field - Bowling Team 

In [ ]:
match_df = match_df.merge(delivery_df, on='ID')

In [ ]:
match_df

In [ ]:
def bowl(row):
    if row['Team1'] == row['BattingTeam']:
        return row['Team2']
    else:
        return row['Team1']

In [ ]:
match_df['BowlingTeam'] = match_df.apply(bowl,axis=1)

In [ ]:
match_df

In [ ]:
delivery_df

In [ ]:
delivery_df = match_df[['ID','innings','overs','ballnumber','batter','bowler','non-striker','extra_type','batsman_run','extras_run','total_run','non_boundary','isWicketDelivery','player_out','kind','fielders_involved','BattingTeam','BowlingTeam']]

In [ ]:
delivery_df

In [ ]:
match_df=match_df[['ID','City','Date','Season','MatchNumber','Team1','Team2','Venue','TossWinner','TossDecision','SuperOver','WinningTeam','WonBy','Margin','method','Player_of_Match']]

In [ ]:
match_df

#### Step 4: Checking the shape of the data

In [ ]:
match_df.shape

In [ ]:
delivery_df.shape

#### Step 5: Group by innings and display the total runs per match in each inning

In [ ]:
total_scores_df = delivery_df.groupby(['ID','innings']).sum()['total_run'].reset_index()

In [ ]:
total_scores_df

#### Step 6: Gather the data of only the first innings

In [ ]:
total_scores_df = total_scores_df[total_scores_df['innings'] == 1]

In [ ]:
total_scores_df

#### Step 7: Merge total_scores_df and match_df

This is done to get the total runs in the match table.

In [ ]:
match_df = match_df.merge(total_scores_df[['ID','total_run']], on='ID')

In [ ]:
match_df

#### Step 8: List the unique teams

In [ ]:
match_df['Team1'].unique()

In [ ]:
teams = [
    'Rajasthan Royals',
    'Royal Challengers Bangalore',
    'Sunrisers Hyderabad',
    'Delhi Capitals',
    'Chennai Super Kings',
    'Gujarat Titans',
    'Lucknow Super Giants',
    'Kolkata Knight Riders',
    'Punjab Kings',
    'Mumbai Indians'
]

In [ ]:
match_df['Team1'] = match_df['Team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['Team2'] = match_df['Team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['Team1'] = match_df['Team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['Team2'] = match_df['Team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

match_df['Team1'] = match_df['Team1'].str.replace('Gujarat Lions','Gujarat Titans')
match_df['Team2'] = match_df['Team2'].str.replace('Gujarat Lions','Gujarat Titans')

match_df['Team1'] = match_df['Team1'].str.replace('Kings XI Punjab','Punjab Kings')
match_df['Team2'] = match_df['Team2'].str.replace('Kings XI Punjab','Punjab Kings')


#### Step 9: Update the match_df

In [ ]:
match_df[match_df['Team1'].isin(teams)]

In [ ]:
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]
match_df = match_df[match_df['WinningTeam'].isin(teams)]

In [ ]:
match_df.shape

In [ ]:
match_df = match_df[['ID','City','WinningTeam','total_run']]

In [ ]:
match_df

#### Step 10: Merge delivery_df and match_df

In [ ]:
delivery_df = match_df.merge(delivery_df,on='ID')

In [ ]:
delivery_df

#### Step 11: Group by second innings

In [ ]:
delivery_df = delivery_df[delivery_df['innings'] == 2]

In [ ]:
delivery_df

#### Step 12: Create remaining fields necessary for prediction

current score

In [ ]:
delivery_df['current_score'] = delivery_df.groupby('ID').cumsum()['total_run_y']

Runs left

In [ ]:
#important because i added +1
delivery_df['runs_left'] = delivery_df['total_run_x'] - delivery_df['current_score']+1

Balls left

In [ ]:
delivery_df['balls_left'] = 126 - (delivery_df['overs']*6 + delivery_df['ballnumber'])

In [ ]:
delivery_df

Wickets

In [ ]:
wickets = delivery_df.groupby('ID').cumsum()['isWicketDelivery'].values
delivery_df['wickets'] = 10 - wickets

In [ ]:
delivery_df

Current Run Rate

In [ ]:
# current_run_rate = runs/overs
delivery_df['current_run_rate'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

Required Run Rate

In [ ]:
# required_run_rate = runs_left/overs_left
delivery_df['required_run_rate'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

Final Result

In [ ]:
def result(row):
    if row['BattingTeam'] == row['WinningTeam']:
        return 1 
    else:
        return 0

In [ ]:
delivery_df['Result'] = delivery_df.apply(result,axis=1)

#### Step 13: Creating the required table

In [ ]:
ultimate_df = delivery_df[['BattingTeam','BowlingTeam','City','runs_left','balls_left','wickets','total_run_x','current_run_rate','required_run_rate','Result']]

In [ ]:
ultimate_df = ultimate_df.sample(ultimate_df.shape[0])

In [ ]:
ultimate_df.sample()

In [ ]:
ultimate_df.isnull().sum()

In [ ]:
ultimate_df.to_csv('IPL_Ball_by_Ball_2008_2022.csv', index=False)